In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Kütüphanelerin okunması
import pandas as pd
import numpy as np
import seaborn as sns
import pandas as pd

# veri dosyasının okunması

df_2010_2011 = pd.read_excel("../input/online-retail-ii-data-set-from-ml-repository/online_retail_II.xlsx", sheet_name = "Year 2010-2011")
df = df_2010_2011.copy()

In [ ]:
# Tüm degisken ve gözlemleri görmek icin
pd.set_option('display.max_columns', None); 
pd.set_option('display.max_rows', None);


# virgulden sonra gösterilecek olan sayı sayısı
pd.set_option('display.float_format', lambda x: '%.2f' % x)
import matplotlib.pyplot as plt

In [ ]:
# Bos deger sorgulama
df.isnull().sum()

In [ ]:
#veri boyutunun sorgusu
df.shape

In [ ]:
# essiz urun sayisi
df.Description.nunique()



In [ ]:
# urunlerin kaç kere gectigini gösteriyor
df.Description.value_counts().head()

In [ ]:
# en cok siparis edilen urun hangisidir ? 
# iptallerden dolayı eksi deger gozukuyor
df.groupby("Description") .agg({"Quantity":"sum"}).head()

In [ ]:
#En cok hangi ürün alınmıs.
df.groupby("Description") .agg({"Quantity":"sum"}).sort_values("Quantity" ,ascending = False).head()



In [ ]:
#Toplamda odenen ucret
df["TotalPrice"] = df["Quantity"] * df["Price"]



In [ ]:
#iptal olanların sorgusu
df[df["Invoice"].str.contains("C" , na = False)].head()

In [ ]:
# iptal olmayan degerleri getirdik
df = df[~df["Invoice"].str.contains("C" , na = False)]



In [ ]:
#Faturaya gore en fazla odenen ucret.
df.groupby("Invoice").agg({"TotalPrice":"sum"}).head()

In [ ]:
# en fazla alışveriş yapan ulke  İngiltere
df.Country.value_counts()[0:5]

In [ ]:
# En fazla para harcayan ulke İngiltere
df.groupby("Country").agg({"TotalPrice":"sum"}).sort_values("TotalPrice" , ascending = False).head()

In [ ]:
# Çeyreklıklerinin gosterilmesi
df.describe([0.01,0.05,0.1,0.25,0.5,0.75,0.9,0.95,0.99]).T

In [ ]:
# Aykırı olan degerlerin gosterimi.
for feature in ["Quantity","Price","TotalPrice"]:

    Q1 = df[feature].quantile(0.01)
    Q3 = df[feature].quantile(0.99)
    IQR = Q3-Q1
    upper = Q3 + 1.5*IQR
    lower = Q1 - 1.5*IQR

    if df[(df[feature] > upper) | (df[feature] < lower)].any(axis=None):
        print(feature,"yes")
        print(df[(df[feature] > upper) | (df[feature] < lower)].shape[0])
    else:
        print(feature, "no")

In [ ]:
# Recency
#En son alısveris yapılan tarih.
df["InvoiceDate"].max()

In [ ]:
# Bugunun tarihi olarak en son gecen tarihi ekledim.
import datetime as dt
today_date = dt.datetime(2011 , 12 ,10)
today_date

In [ ]:
# Musterilerin en son ne zaman alısveris yaptıga bakıyoruz.
df.groupby("Customer ID").agg({"InvoiceDate":"max"}).head()

In [ ]:
# guzel gorunsun diye boyle yaptık
df["Customer ID"] = df["Customer ID"].dropna().astype(int)

In [ ]:
# Musterilerin en son ne zaman alısveris yaptıga bakıyoruz.
df.groupby("Customer ID").agg({"InvoiceDate":"max"}).head()

In [ ]:
#Musterilerin en son alısveris yaptıgı tarihten bugunu cıkarıyoruz.
(today_date - df.groupby("Customer ID").agg({"InvoiceDate":"max"})).head()

In [ ]:
temp_df = (today_date - df.groupby("Customer ID").agg({"InvoiceDate":"max"}))

In [ ]:
#Degisken ismini degistirdim.
temp_df.rename(columns = {"InvoiceDate":"Recency"} , inplace= True)



In [ ]:
#Sadece gün sayısını elde ettik.
recency_df = temp_df.Recency.apply(lambda x : x.days)

In [ ]:
# Frequency (sıklık)
df.groupby(["Customer ID", "Invoice"]).agg({"Invoice":"nunique"}).head(4)

In [ ]:
freq_df = df.groupby("Customer ID").agg({"InvoiceDate":"nunique"})



In [ ]:
#Degisken ismini degistirdim.
freq_df.rename(columns = {"InvoiceDate" : "Frequency"} , inplace = True)

In [ ]:
# Monetary (musterinin toplam bıraktıgı para mıktarı)
monetary_df = df.groupby("Customer ID").agg({"TotalPrice":"sum"})
monetary_df.head()

In [ ]:
#Degisken ismini degistirdim.
monetary_df.rename(columns= {"TotalPrice":"Monetary"} , inplace = True)

In [ ]:
#Degiskenlerin birlestirilmesi
rfm = pd.concat([recency_df , freq_df , monetary_df] , axis = 1)



In [ ]:
# Skorlama işlemi
rfm["RecencyScore"] = pd.qcut(rfm["Recency"], 5, labels = [5, 4 , 3, 2, 1])
rfm.head()
rfm["Recency"].min()
rfm["Frequency"].min()

In [ ]:
# Skorlama işlemi
rfm["FrequencyScore"]= pd.qcut(rfm["Frequency"].rank(method="first"),5, labels=[1,2,3,4,5])

In [ ]:
# Skorlama işlemi
rfm["MonetaryScore"] = pd.qcut(rfm["Monetary"] , 5 , labels = [1,2,3,4,5])



In [ ]:
# rfm skorları kategorik degere donusturuldu
rfm["rfm_score"] = (rfm.RecencyScore.astype(str)+
                    rfm.FrequencyScore.astype(str) +
                    rfm.MonetaryScore.astype(str))



In [ ]:
#En cok sipariş veren ve sitede aktif olan kişilerin getirlmesi.
rfm[rfm["rfm_score"]=="555"].head()

In [ ]:
#Betimleyici istatistiklerinin getirilmesi.
rfm.describe().T

In [ ]:
#uykuda olanlar
rfm[rfm["rfm_score"] == "111"].head()

In [ ]:
# Regular Expressions (Düzenli İfadeler) kullanılarak RFM haritası çıkarıldı
seg_map = {
    r'[1-2][1-2]': 'Hibernating',
    r'[1-2][3-4]': 'At Risk',
    r'[1-2]5': 'Can\'t Loose',
    r'3[1-2]': 'About to Sleep',
    r'33': 'Need Attention',
    r'[3-4][4-5]': 'Loyal Customers',
    r'41': 'Promising',
    r'51': 'New Customers',
    r'[4-5][2-3]': 'Potential Loyalists',
    r'5[4-5]': 'Champions'
}

In [ ]:
#Monetary degeri dısarıda bırakılarak ,  FrequencyScore ve RecencyScore degerlerinin birlestirilerek segment isminde degisken olusturulması.
rfm["segment"] =rfm.RecencyScore.astype(str) + rfm.FrequencyScore.astype(str)
rfm.head() 

In [ ]:
df_rfm = rfm[["RecencyScore" , "FrequencyScore" ,"MonetaryScore"]]
df_rfm.head()

In [ ]:
#OLusturdugumuz regex sozlugunun tum segment'e uygulanarak isimlendirdim.
rfm.segment = rfm.segment.replace(seg_map , regex = True)

In [ ]:
#df ve rfm'in birlestilmesi
result = pd.merge(df, rfm, on='Customer ID')

In [ ]:
#Segment'e gore grupların toplam harcamasının medyan degeri.
result.groupby("segment").agg({"TotalPrice":np.median})

In [ ]:
#Ayların eklenmesi.
result['month'] = result['InvoiceDate'].dt.month



In [ ]:
result_bir = result.loc[(result["segment"] == "Need Attention")]
result_iki = result.loc[(result["segment"] == "Hibernating")]
result_uc = result.loc[(result["segment"] == "At Risk")]     
dff = pd.concat([result_bir , result_iki , result_uc] , axis = 0)                  

In [ ]:
#Aylara gore sıklıgın , segment kırılımında incelenmesi
sns.lineplot(x = "month", y = "Frequency", hue = "segment", data = dff);

In [ ]:
# Aylara gore toplam harcamanın , segment kırılımında incelenmesi.
sns.lineplot(x = "month", y = "TotalPrice", hue = "segment", data = dff);

In [ ]:
#Belirlediğimiz 3 segmente gore sıklıgın incelenmesi.
sns.barplot(x = "segment", y = "Frequency", data = dff)

In [ ]:
# Segmentlerin recency , frequency ve monetary degerlerine gore ortalama ve medyanın incelenmesi.
rfm[["segment","Recency","Frequency","Monetary"]].groupby("segment").agg(["mean","median","count"])

In [ ]:
# Segmentlerin recency , frequency ve monetary degerlerine gore ortalama ve medyanın incelenmesi.
rfm[["segment","Recency","Frequency","Monetary"]].groupby("segment").agg(["mean","median","count"])

In [ ]:
#KMEANS KUMELEME

import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from yellowbrick.cluster import KElbowVisualizer
from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import dendrogram

import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# EDA



df = df_rfm.copy()
df.head()
df.isnull().sum()
df.info()
df.describe().T




# K-MEANS
kmeans = KMeans(n_clusters=10)
k_fit = kmeans.fit(df)
k_fit


k_fit.n_clusters
k_fit.cluster_centers_
k_fit.labels_
df[0:5]

# Kümelerin Görselleştirilmesi
k_means = KMeans(n_clusters=10).fit(df)
kumeler = k_means.labels_
type(df)
df = pd.DataFrame(df)

plt.scatter(df.iloc[:, 0],
            df.iloc[:, 1],
            c=kumeler,
            s=50,
            cmap="viridis")
plt.show()


# merkezlerin isaretlenmesi
merkezler = k_means.cluster_centers_

plt.scatter(df.iloc[:, 0],
            df.iloc[:, 1],
            c=kumeler,
            s=50,
            cmap="viridis")

plt.scatter(merkezler[:, 0],
            merkezler[:, 1],
            c="black",
            s=200,
            alpha=0.5)
plt.show()


# Optimum Küme Sayısının Belirlenmesi
kmeans = KMeans()
ssd = []
K = range(1, 30)

for k in K:
    kmeans = KMeans(n_clusters=k).fit(df)
    ssd.append(kmeans.inertia_)

ssd

plt.plot(K, ssd, "bx-")
plt.xlabel("Farklı K Değerlerine Karşılık Uzaklık Artık Toplamları")
plt.title("Optimum Küme sayısı için Elbow Yöntemi")
plt.show()


In [ ]:
?k_means

In [ ]:


# Daha otomatik bir yol:
df = df_rfm.copy()
kmeans = KMeans()
#visu = KElbowVisualizer(kmeans, k=(2, 20))
#visu.fit(df)
#visu.show()


kmeans = KMeans(n_clusters=10).fit(df)
kumeler = kmeans.labels_



pd.DataFrame({"musteri_no": df.index, "Kumeler": kumeler})

df["cluster_no"] = kumeler

df.head()

df["cluster_no"] = df["cluster_no"] + 1

df.groupby("cluster_no").agg({"cluster_no": "count"})
#df.groupby("cluster_no").agg(np.mean)

df[df["cluster_no"] == 4].head()

